In [1]:
import json
from gensim import corpora
import numpy as np

with open('profiles/profiles_adj/topicTree.nodes.json', 'r') as read_file:
    adj = json.load(read_file)
with open('profiles/profiles_loc/topicTree.nodes.json', 'r') as read_file:
    loc = json.load(read_file)
with open('profiles/profiles_misc/topicTree.nodes.json', 'r') as read_file:
    misc = json.load(read_file)
with open('profiles/profiles_noun/topicTree.nodes.json', 'r') as read_file:
    noun = json.load(read_file)
with open('profiles/profiles_org/topicTree.nodes.json', 'r') as read_file:
    org = json.load(read_file)
with open('profiles/profiles_per/topicTree.nodes.json', 'r') as read_file:
    per = json.load(read_file)
with open('profiles/profiles_propn/topicTree.nodes.json', 'r') as read_file:
    propn = json.load(read_file)
with open('profiles/profiles_verb/topicTree.nodes.json', 'r') as read_file:
    verb = json.load(read_file)
tree_types = [adj, loc, misc, noun, org, per, propn, verb]

In [2]:
topic_words = []

def dfs(node):
    children = node['children']
    for child in children:
        topic_words.append(child['text'].split(' '))
        dfs(child)

for i in tree_types:
    for j in i:
        topic_words.append(j['text'].split(' '))
        dfs(j)

In [4]:
dic = corpora.Dictionary(topic_words)
print(dic.num_docs)
print(dic.num_pos)

733
4757


In [5]:
word_dict = {}
for i in dic.iteritems():
    word_dict[i[1]] = i[0]

In [7]:
def dfs(node, graph, parents):
    for p in parents:
        for p2 in parents:
            if p != p2:
                graph[p][p2] = 2

    children = node['children']
    for child in children:
        child_words = []
        for h in child['text'].split(' '):
            child_words.append(word_dict[h])
        for p in parents:
            for k in child_words:
                graph[p][k] = 1
        dfs(child, graph, child_words)

In [8]:
graphs = []
current_graph = np.zeros((dic.num_pos, dic.num_pos))
graphs.append(current_graph)
for i in tree_types:
    for j in i:
        current_parents = []
        for word in j['text'].split(' '):
            current_parents.append(word_dict[word])
        dfs(j, current_graph, current_parents)

In [ ]:
graphs = []
for i in tree_types:
    current_graph = np.zeros((dic.num_pos, dic.num_pos))
    graphs.append(current_graph)
    for j in i:
        current_parents = []
        for word in j['text'].split(' '):
            current_parents.append(word_dict[word])
        dfs(j, current_graph, current_parents)

In [10]:
i = 0
for g in graphs:
    with open("single_matrix/view" + str(i) + ".npy", 'wb') as f:
        np.save(f, g)
    i += 1